In [158]:
import boto3
import pandas as pd
from io import StringIO

In [159]:
AWS_ACCESS_KEY = ""
AWS_SECRET_KEY = ""
AWS_REGION = "us-east-1"
SCHEMA_NAME = "childcare_population_dataset"
S3_STAGING_DIR = "s3://athena-output-childcare-population-dataset/output/"
S3_BUCKET_NAME = "athena-output-childcare-population-dataset"
S3_OUTPUT_DIRECTORY = "output"

In [160]:
athena_client = boto3.client(
'athena',
aws_access_key_id=AWS_ACCESS_KEY,
aws_secret_access_key=AWS_SECRET_KEY,
region_name=AWS_REGION,
)

In [161]:
Dict = {}
def download_and_load_query_results(
client: boto3.client, query_response: Dict
) -> pd.DataFrame:
    while True:
        try:
            client.get_query_results(
                QueryExecutionId=query_response['QueryExecutionId']
            )
            break
        except Exception as err:
            if "not yet finished" in str(err):
                time.sleep(0.001)
            else:
                raise err
    temp_file_location: str ="childcare_population_results.csv"
    s3_client = boto3.client(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=AWS_REGION,
    )
    s3_client.download_file(
        S3_BUCKET_NAME,
        f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv",
        temp_file_location,
    )
    return pd.read_csv(temp_file_location)


In [162]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM de_childcare_providers",
    QueryExecutionContext={
        "Database": SCHEMA_NAME
        },
    ResultConfiguration={
        'OutputLocation': S3_STAGING_DIR,
        "EncryptionConfiguration":{"EncryptionOption": "SSE_S3"},
    },
)   

In [163]:
response

{'QueryExecutionId': '54ea5447-eef0-4882-8baa-ed852f7e8e80',
 'ResponseMetadata': {'RequestId': 'caa9642e-3e48-4453-b24f-da47eff15e61',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sun, 24 Jul 2022 17:49:35 GMT',
   'x-amzn-requestid': 'caa9642e-3e48-4453-b24f-da47eff15e61',
   'content-length': '59',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [164]:
de_childcare_providers = download_and_load_query_results(athena_client, response)

In [165]:
de_childcare_providers.head()

,license_number,county,type_of_child_care,city,state,age_range,capacity,count
0,27390,Sussex,Licensed Family Child Care,GEORGETOWN,DE,6 months through 12 years.,9,1
1,27399,New Castle,Licensed Child Care Center,WILMINGTON,DE,6 weeks through 5 years.,99,1
2,27407,Kent,Licensed Child Care Center,DOVER,DE,1 year through 12 years.,59,1
3,27410,New Castle,Licensed Family Child Care,WILMINGTON,DE,6 weeks through 12 years.,9,1
4,27411,New Castle,Licensed Family Child Care,WILMINGTON,DE,6 weeks through 12 years.,9,1


In [166]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM population_stats_for_de_2015_2016_5y___under_5",
    QueryExecutionContext={
        "Database": SCHEMA_NAME
        },
    ResultConfiguration={
        'OutputLocation': S3_STAGING_DIR,
        "EncryptionConfiguration":{"EncryptionOption": "SSE_S3"},
    },
)   

In [167]:
population_stats_for_de_2015_2016_5y___under_5 = download_and_load_query_results(athena_client, response)

In [168]:
population_stats_for_de_2015_2016_5y___under_5.head()

,county,blank,total_pop,total_pop_under_5
0,"""Kent County",NaN,179124,11091
1,"""New Castle County",NaN,558306,32070
2,"""Sussex County",NaN,230249,11613


In [169]:
combined_data_1 = de_childcare_providers[['license_number', 'county', 'type_of_child_care', 'city', 'age_range', 'capacity', 'count']]
combined_data_2 = population_stats_for_de_2015_2016_5y___under_5[['county', 'total_pop', 'total_pop_under_5']]
combined_data = pd.merge(combined_data_1, combined_data_2, how='outer' )

In [170]:
combined_data.head()

,license_number,county,type_of_child_care,city,age_range,capacity,count,total_pop,total_pop_under_5
0,27390.0,Sussex,Licensed Family Child Care,GEORGETOWN,6 months through 12 years.,9.0,1.0,NaN,NaN
1,27473.0,Sussex,Licensed Family Child Care,MILTON,6 weeks through 12 years.,9.0,1.0,NaN,NaN
2,27501.0,Sussex,Licensed Family Child Care,MILLSBORO,6 weeks through 12 years.,9.0,1.0,NaN,NaN
3,27923.0,Sussex,Licensed Family Child Care,FRANKFORD,6 weeks through 8 years.,9.0,1.0,NaN,NaN
4,27929.0,Sussex,Licensed Family Child Care,FRANKFORD,6 weeks through 3 years.,9.0,1.0,NaN,NaN


In [171]:
bucket = 'athena-output-childcare-population-dataset'

In [172]:
csv_buffer = StringIO()


In [173]:
combined_data.to_csv(csv_buffer)


In [174]:
s3_resource = boto3.resource(
    's3',
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=AWS_REGION,
)


In [175]:
s3_resource.Object(bucket, 'output/combined_data.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'AXE6EY8P00X9MNRV',
  'HostId': 'aqw3kGGDVIjy1opC4OaxbixFHAIltZiwmFYsMh9kTMyp6SN9nvdv9qQDbYfdFDJhqbtoC3uW4O4=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'aqw3kGGDVIjy1opC4OaxbixFHAIltZiwmFYsMh9kTMyp6SN9nvdv9qQDbYfdFDJhqbtoC3uW4O4=',
   'x-amz-request-id': 'AXE6EY8P00X9MNRV',
   'date': 'Sun, 24 Jul 2022 17:50:06 GMT',
   'etag': '"5e10c6957d22e2249fcbf4d211219fb7"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"5e10c6957d22e2249fcbf4d211219fb7"'}

In [176]:
bucket = 'athena-output-childcare-population-dataset'
csv_buffer = StringIO()
combined_data_1.to_csv(csv_buffer)
s3_resource = boto3.resource(
    's3',
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=AWS_REGION,
)
s3_resource.Object(bucket, 'output/combined_data_1.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'XB69T0EMHN95VVQG',
  'HostId': 'Ck4TeMiAx7Dc44ihwxxwDUHn/po7KRlU6ApYe3Ly47m5AlaoxFY/YhXf/AuqAqKlqmqgRPn+rGg=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'Ck4TeMiAx7Dc44ihwxxwDUHn/po7KRlU6ApYe3Ly47m5AlaoxFY/YhXf/AuqAqKlqmqgRPn+rGg=',
   'x-amz-request-id': 'XB69T0EMHN95VVQG',
   'date': 'Sun, 24 Jul 2022 17:50:08 GMT',
   'etag': '"0fe67a1642844259a2c0f7b96e908fbe"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"0fe67a1642844259a2c0f7b96e908fbe"'}

In [177]:
bucket = 'athena-output-childcare-population-dataset'
csv_buffer = StringIO()
combined_data_2.to_csv(csv_buffer)
s3_resource = boto3.resource(
    's3',
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=AWS_REGION,
)
s3_resource.Object(bucket, 'output/combined_data_2.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'ATBRKRD6YGQ63ZK5',
  'HostId': 'DywoPHawOzUBMQKPviq0BLpDruwVtowgN7H/YnahibxYs1UuUVg47hpw8N1auv0gsmTV7mDUro4=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'DywoPHawOzUBMQKPviq0BLpDruwVtowgN7H/YnahibxYs1UuUVg47hpw8N1auv0gsmTV7mDUro4=',
   'x-amz-request-id': 'ATBRKRD6YGQ63ZK5',
   'date': 'Sun, 24 Jul 2022 17:50:10 GMT',
   'etag': '"3b93361f27b4b29b1a6a739b24a91859"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"3b93361f27b4b29b1a6a739b24a91859"'}

In [178]:
combined_data_1sql = pd.io.sql.get_schema(combined_data_1.reset_index(), 'combined_data_1')
print(''.join(combined_data_1sql))

CREATE TABLE "combined_data_1" (
"index" INTEGER,
  "license_number" INTEGER,
  "county" TEXT,
  "type_of_child_care" TEXT,
  "city" TEXT,
  "age_range" TEXT,
  "capacity" INTEGER,
  "count" INTEGER
)


In [179]:
combined_data_2sql = pd.io.sql.get_schema(combined_data_2.reset_index(), 'combined_data_2')
print(''.join(combined_data_2sql))

CREATE TABLE "combined_data_2" (
"index" INTEGER,
  "county" TEXT,
  "total_pop" INTEGER,
  "total_pop_under_5" INTEGER
)


In [180]:
combined_datasql = pd.io.sql.get_schema(combined_data.reset_index(), 'combined_data')
print(''.join(combined_datasql))

CREATE TABLE "combined_data" (
"index" INTEGER,
  "license_number" REAL,
  "county" TEXT,
  "type_of_child_care" TEXT,
  "city" TEXT,
  "age_range" TEXT,
  "capacity" REAL,
  "count" REAL,
  "total_pop" REAL,
  "total_pop_under_5" REAL
)


In [181]:
import redshift_connector

In [185]:
conn = redshift_connector.connect(
host='redshift-cluster-1.cipnettqiss0.us-east-1.redshift.amazonaws.com',
database='dev',
user='awsuser',
password='Password123!'
 )
           
                 

In [186]:
conn.autocommit = True

In [192]:
cursor = redshift_connector.Cursor = conn.cursor()
cursor.execute("""
               CREATE TABLE "combined_data_1" (
"index" INTEGER,
  "license_number" INTEGER,
  "county" TEXT,
  "type_of_child_care" TEXT,
  "city" TEXT,
  "age_range" TEXT,
  "capacity" INTEGER,
  "count" INTEGER
)
""")

In [193]:

cursor.execute("""
               CREATE TABLE "combined_data_2" (
"index" INTEGER,
  "county" TEXT,
  "total_pop" INTEGER,
  "total_pop_under_5" INTEGER
)
""")

In [194]:
cursor.execute("""
               CREATE TABLE "combined_data" (
"index" INTEGER,
  "license_number" REAL,
  "county" TEXT,
  "type_of_child_care" TEXT,
  "city" TEXT,
  "age_range" TEXT,
  "capacity" REAL,
  "count" REAL,
  "total_pop" REAL,
  "total_pop_under_5" REAL
)
""")

In [201]:
cursor.execute("""
copy combined_data_1 from 's3://athena-output-childcare-population-dataset/output/combined_data_1.csv'
credentials 'aws_iam_role=arn:aws:iam::298203450806:role/redshift-s3-access'
delimiter ','
region 'us-east-1'
IGNOREHEADER 1
""")

In [202]:
cursor.execute("""
copy combined_data_2 from 's3://athena-output-childcare-population-dataset/output/combined_data_2.csv'
credentials 'aws_iam_role=arn:aws:iam::298203450806:role/redshift-s3-access'
delimiter ','
region 'us-east-1'
IGNOREHEADER 1
""")